In [ ]:
import sys
import os
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from utilities import create_model
from utilities import TrainerConfiguration
from utilities import TrainDataset

In [ ]:
BATCH_SIZE=128
KERNEL_SIZE=4
ACTIVATION_FUNCTION='relu'
LEARNING_RATE=0.001
NUM_UNITS_DENSE=256
NUM_UNITS_LTSM2=1024
### run 55 parameters
DROPOUT=0.12489316869910207
NUM_UNITS_LTSM1=512
### run 41 parameters
# DROPOUT=0.1
# NUM_UNITS_LTSM1=768

In [ ]:
# Load a full model, previously trained
model_path = Path('./10-iam_train-run_55-standley_train-retrained_100e_01lr-full_model/')
full_model = keras.models.load_model(model_path)

In [ ]:
# OR, load a PREDICTION model and build a new CTC layer
model_path = 'gs://iam-model-staging/run_55_train/model/'
pred_model = keras.models.load_model(model_path)
pred_model.save_weights(Path('checkpoints', 'pred'))

full_model = create_model(KERNEL_SIZE, ACTIVATION_FUNCTION, NUM_UNITS_DENSE, DROPOUT, NUM_UNITS_LTSM1, NUM_UNITS_LTSM2, LEARNING_RATE)
full_model.load_weights(Path('checkpoints', 'pred'))

In [ ]:
# load Steyermark or Standley images
folder = Path('images/standley_4058_train')
metadata = Path('words_metadata.csv')
data = TrainDataset()
data.create_dataset(BATCH_SIZE, folder, metadata)

In [ ]:
# RETRAIN the model -- fit only on open layers
for i in range(len(full_model.layers)):
    full_model.layers[i].trainable=False
full_model.layers[6].trainable = True
full_model.layers[-1].trainable = True
full_model.compile(keras.optimizers.Adam(learning_rate=LEARNING_RATE))

history = full_model.fit(data.train_dataset, epochs=100, validation_data=data.validation_dataset)

full_model.save('retrained-full_model')

prediction_model = tf.keras.models.Model(
    full_model.get_layer(name='image').input, full_model.get_layer(name='dense_layer').output
)
prediction_model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
prediction_model.save('retrained')

In [ ]:
# Then, fine tune the model
full_model.trainable = True
full_model.compile(keras.optimizers.Adam(learning_rate=1e-5))
history_fine_tune = full_model.fit(data.train_dataset, epochs=100, validation_data=data.validation_dataset)


full_model.save('fine_tuned-full_model')

prediction_model = tf.keras.models.Model(
    full_model.get_layer(name='image').input, full_model.get_layer(name='dense_layer').output
)
prediction_model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
prediction_model.save('fine_tuned')